# University Programs Data Preprocessing and Merging

## Overview

This notebook performs comprehensive data preprocessing and merging of two educational datasets to create a unified, cleaned dataset for use in the DEPI chatbot system.

## What This Notebook Does

### 1. **Data Loading**
   - Loads university programs data from `all_programs.xlsx` (14,230 initial records)
   - Loads Indian college data from `college_data.csv` (11,800 initial records)

### 2. **First Dataset Processing (all_programs.xlsx)**
   - **Data Cleaning**: Removes rows with missing IELTS, TOEFL, or GPA scores
   - **Feature Selection**: Keeps only English columns and essential features (university name, program, language, duration, IELTS, TOEFL, GPA, price)
   - **Fee Calculation**: Extracts course duration and calculates total fees (price × duration)
   - **Column Standardization**: Renames columns for consistency (e.g., `ilts` → `ielts`, `nameEn` → `program`)
   - **Result**: 13,130 cleaned records

### 3. **Second Dataset Processing (college_data.csv)**
   - **Data Cleaning**: Removes rows with missing program or fees information
   - **Currency Conversion**: Converts fees from Indian Rupees (INR) to USD (exchange rate: 83 INR = 1 USD)
   - **Data Cleaning**: Removes currency symbols, commas, and formatting from fees column
   - **Column Standardization**: Renames `college_name` to `university_name` to match first dataset
   - **Result**: 5,466 cleaned records

### 4. **Data Merging**
   - Standardizes university names (lowercase, trimmed) for matching
   - Performs outer join on: `university_name`, `program`, `fees`, and `duration`
   - **Result**: 18,596 merged records combining both datasets

### 5. **Final Data Cleaning**
   - Fills missing IELTS and TOEFL scores with their respective mean values
   - Saves the final cleaned dataset to `all_programs_cleaned.xlsx`

## Output

The final dataset contains:
- **18,596 program records** from universities worldwide
- **9 columns**: program, course_languageEn, duration, university_name, ielts, toefl, gpa, fees, college_link
- **Standardized format** ready for chatbot integration


## 1. Import Required Libraries


In [46]:
import numpy as np
import pandas as pd
import os

## 2. Load and Explore First Dataset

Load the university programs dataset from Excel file and examine its structure.


In [103]:
df1 = pd.read_excel('..\\data\\all_programs.xlsx')


# Explore the data
print("Shape:", df1.shape)
print("\nColumns:", df1.columns.tolist())
print("\nFirst few rows:")
print(df1[:30])
print("\nData types:")
print(df1.dtypes)
print("\nMissing values:")
print(df1.isnull().sum())

Shape: (14230, 25)

Columns: ['university_name', 'nameEn', 'nameAr', 'major_name', 'field_name', 'academic_degreeEn', 'academic_degreeAr', 'course_languageEn', 'course_languageAr', 'attendance_methodEn', 'attendance_methodAr', 'course_durationEn', 'course_durationAr', 'price', 'price_period', 'price_period_typeEn', 'price_period_typeAr', 'ilts', 'toefl', 'gpa', 'bank_statement', 'start_date', 'end_date', 'recommend', 'periodic_notes']

First few rows:
        university_name                                             nameEn  \
0    Academy of Silesia                                 BA Interior Design   
1    Academy of Silesia                                 MA Interior Design   
2    Academy of Silesia                                    BA Architecture   
3    Academy of Silesia                                   MA Architecture    
4    Academy of Silesia                             BSc Civil Construction   
5    Academy of Silesia                             MSc Civil Construction  

## 3. Clean and Transform First Dataset

### 3.1 Remove Missing Values
Remove rows with missing IELTS, TOEFL, or GPA scores.

### 3.2 Feature Selection
Select only English columns and essential features (university name, IELTS, TOEFL, GPA, price).

### 3.3 Calculate Total Fees
Extract duration from course_durationEn and calculate total fees (price × duration).

### 3.4 Rename and Drop Columns
Standardize column names and remove unnecessary columns.


In [104]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14230 entries, 0 to 14229
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   university_name      14230 non-null  object 
 1   nameEn               14230 non-null  object 
 2   nameAr               14140 non-null  object 
 3   major_name           14230 non-null  object 
 4   field_name           14230 non-null  object 
 5   academic_degreeEn    14230 non-null  object 
 6   academic_degreeAr    14141 non-null  object 
 7   course_languageEn    14230 non-null  object 
 8   course_languageAr    14141 non-null  object 
 9   attendance_methodEn  14230 non-null  object 
 10  attendance_methodAr  14141 non-null  object 
 11  course_durationEn    14230 non-null  object 
 12  course_durationAr    14141 non-null  object 
 13  price                14230 non-null  int64  
 14  price_period         14230 non-null  int64  
 15  price_period_typeEn  14230 non-null 

In [105]:

df1[["ilts", "toefl", "gpa"]] = df1[["ilts", "toefl", "gpa"]].replace(
    ["", " ", "NA", "None", "-"], np.nan
)
df1 = df1.dropna(subset=["ilts", "toefl", "gpa"])

In [106]:
df1_final = df1[[col for col in df1.columns if col.endswith("En")]+["university_name"]+["ilts"]+["toefl"]+["gpa"]+["price"]]
df1_final.head()

,nameEn,academic_degreeEn,course_languageEn,attendance_methodEn,course_durationEn,price_period_typeEn,university_name,ilts,toefl,gpa,price
66,BA Graphic design,Bachelor's,English,On Campus,3 Years,Year,Budapest Metropolitan University,5.0,69.0,Not Required,9980
67,BSc in Hotel and Tourism Management,Bachelor's,English,On Campus,3 Years,Year,Cesar Ritz Colleges,5.0,55.0,Not Required,22000
68,BA in Hospitality Business Management,Bachelor's,English,On Campus,3 Years,Year,Cesar Ritz Colleges,5.0,55.0,Not Required,27900
69,MSc in Leadership,Master,English,On Campus,1 Year,Year,Cesar Ritz Colleges,5.0,55.0,Required,33000
70,MA in Hospitality and Tourism Entrepreneurship,Master,English,On Campus,1 Year,Year,Cesar Ritz Colleges,5.0,55.0,Required,29300


In [107]:
duration = df1_final["course_durationEn"].str.extract(r'(\d+)')[0].astype(float)
df1_final["fees"] = df1_final["price"] * duration


C:\Users\DELL\AppData\Local\Temp\ipykernel_3896\968366838.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_final["fees"] = df1_final["price"] * duration


In [108]:
df1_final = df1_final.rename(columns={
    "nameEn": "program",
    "course_durationEn": "duration",
    "ilts": "ielts"
})
df1_final = df1_final.drop(columns=["attendance_methodEn","academic_degreeEn","price_period_typeEn","price"])


## 4. Load and Explore Second Dataset

Load the Indian college dataset from CSV file and examine its structure.


,program,course_languageEn,duration,university_name,ielts,toefl,gpa,fees
66,BA Graphic design,English,3 Years,Budapest Metropolitan University,5.0,69.0,Not Required,29940.0
67,BSc in Hotel and Tourism Management,English,3 Years,Cesar Ritz Colleges,5.0,55.0,Not Required,66000.0
68,BA in Hospitality Business Management,English,3 Years,Cesar Ritz Colleges,5.0,55.0,Not Required,83700.0
69,MSc in Leadership,English,1 Year,Cesar Ritz Colleges,5.0,55.0,Required,33000.0
70,MA in Hospitality and Tourism Entrepreneurship,English,1 Year,Cesar Ritz Colleges,5.0,55.0,Required,29300.0


In [78]:

try:
    df2 = pd.read_csv('..\\data\\college_data.csv')

    print("DataFrame created successfully from file:")
    print(df2)

except FileNotFoundError:
    print(f"Error: The file was not found at the specified path: {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")


DataFrame created successfully from file:
                                            college_name  \
0           Indian Institute of Technology (IIT), Kanpur   
1                Indian Institute of Technology, Chennai   
2        Indian Institute of Technology (IIT), Kharagpur   
3          Indian Institute of Technology Bombay, Mumbai   
4          Indian Institute of Technology (IIT), Roorkee   
...                                                  ...   
11795                               ITM Aligarh, Aligarh   
11796                    Motherhood University, Haridwar   
11797       Online Vivekananda Global University, Jaipur   
11798  Indian School of Science and Management (ISSM)...   
11799    Indian Institute of Management Sirmaur, Sirmaur   

                                            college_link duration  \
0      https://www.collegesearch.in/colleges/indian-i...  4 Years   
1      https://www.collegesearch.in/colleges/indian-i...  4 Years   
2      https://www.collegesear

## 5. Clean and Transform Second Dataset

### 5.1 Remove Missing Values
Remove rows with missing program or fees information.

### 5.2 Convert Fees to USD
Clean the fees column (remove "Rs", "/-", commas) and convert from Indian Rupees to USD (exchange rate: 83 INR = 1 USD).

### 5.3 Standardize Column Names
Rename columns to match the first dataset format (college_name → university_name) and drop unnecessary columns.


In [79]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11800 entries, 0 to 11799
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   college_name  11800 non-null  object 
 1   college_link  11800 non-null  object 
 2   duration      11800 non-null  object 
 3   fees          5476 non-null   object 
 4   rank          0 non-null      float64
 5   placement     1388 non-null   object 
 6   c_type        1791 non-null   object 
 7   program       11770 non-null  object 
dtypes: float64(1), object(7)
memory usage: 737.6+ KB


In [80]:
df2[["program", "fees"]] = df2[["program", "fees"]].replace(
    ["", " ", "NA", "None", "-"], np.nan
)
df2 = df2.dropna(subset=["program", "fees"])

In [81]:
df2.head()

,college_name,college_link,duration,fees,rank,placement,c_type,program
0,"Indian Institute of Technology (IIT), Kanpur",https://www.collegesearch.in/colleges/indian-i...,4 Years,"Rs 8,76,186/-",NaN,Average -Rs.\r\n ...,Autonomous,Bachelor of Technology (B.Tech)
1,"Indian Institute of Technology, Chennai",https://www.collegesearch.in/colleges/indian-i...,4 Years,"Rs 8,74,950/-",NaN,Average -Rs.\r\n ...,Autonomous,Bachelor of Technology (B.Tech)
2,"Indian Institute of Technology (IIT), Kharagpur",https://www.collegesearch.in/colleges/indian-i...,4 Years,"Rs 10,65,980/-",NaN,Average -Rs.\r\n ...,Autonomous,Bachelor of Technology (B.Tech)
3,"Indian Institute of Technology Bombay, Mumbai",https://www.collegesearch.in/colleges/indian-i...,4 Years,"Rs 9,12,000/-",NaN,Average -Rs.\r\n ...,Autonomous,Bachelor of Technology (B.Tech)
4,"Indian Institute of Technology (IIT), Roorkee",https://www.collegesearch.in/colleges/indian-i...,4 Years,"Rs 6,28,120/-",NaN,Average -Rs.\r\n ...,Autonomous,Bachelor of Technology (B.Tech)


In [82]:
df2["fees"] = (
    df2["fees"]
    .str.replace("Rs", "", regex=False)
    .str.replace("/-", "", regex=False)
    .str.replace(",", "", regex=False)
    .str.strip()
)

df2["fees"] = df2["fees"].astype(float)
df2["fees_usd"] = df2["fees"] / 83 
df2["fees"] = df2["fees_usd"].round(2)
df2.head()


,college_name,college_link,duration,fees,rank,placement,c_type,program,fees_usd
0,"Indian Institute of Technology (IIT), Kanpur",https://www.collegesearch.in/colleges/indian-i...,4 Years,10556.46,NaN,Average -Rs.\r\n ...,Autonomous,Bachelor of Technology (B.Tech),10556.457831
1,"Indian Institute of Technology, Chennai",https://www.collegesearch.in/colleges/indian-i...,4 Years,10541.57,NaN,Average -Rs.\r\n ...,Autonomous,Bachelor of Technology (B.Tech),10541.566265
2,"Indian Institute of Technology (IIT), Kharagpur",https://www.collegesearch.in/colleges/indian-i...,4 Years,12843.13,NaN,Average -Rs.\r\n ...,Autonomous,Bachelor of Technology (B.Tech),12843.132530
3,"Indian Institute of Technology Bombay, Mumbai",https://www.collegesearch.in/colleges/indian-i...,4 Years,10987.95,NaN,Average -Rs.\r\n ...,Autonomous,Bachelor of Technology (B.Tech),10987.951807
4,"Indian Institute of Technology (IIT), Roorkee",https://www.collegesearch.in/colleges/indian-i...,4 Years,7567.71,NaN,Average -Rs.\r\n ...,Autonomous,Bachelor of Technology (B.Tech),7567.710843


In [86]:
df2_final = df2.drop(columns=["fees_usd","c_type","placement","rank"])
df2_final = df2_final.rename(columns={"college_name": "university_name"})
df2_final.head()

,university_name,college_link,duration,fees,program
0,"Indian Institute of Technology (IIT), Kanpur",https://www.collegesearch.in/colleges/indian-i...,4 Years,10556.46,Bachelor of Technology (B.Tech)
1,"Indian Institute of Technology, Chennai",https://www.collegesearch.in/colleges/indian-i...,4 Years,10541.57,Bachelor of Technology (B.Tech)
2,"Indian Institute of Technology (IIT), Kharagpur",https://www.collegesearch.in/colleges/indian-i...,4 Years,12843.13,Bachelor of Technology (B.Tech)
3,"Indian Institute of Technology Bombay, Mumbai",https://www.collegesearch.in/colleges/indian-i...,4 Years,10987.95,Bachelor of Technology (B.Tech)
4,"Indian Institute of Technology (IIT), Roorkee",https://www.collegesearch.in/colleges/indian-i...,4 Years,7567.71,Bachelor of Technology (B.Tech)


### Verify Dataset Structures Before Merging


In [110]:
df1_final.info()
df2_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13130 entries, 66 to 14229
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   program            13130 non-null  object 
 1   course_languageEn  13130 non-null  object 
 2   duration           13130 non-null  object 
 3   university_name    13130 non-null  object 
 4   ielts              13130 non-null  float64
 5   toefl              13130 non-null  float64
 6   gpa                13130 non-null  object 
 7   fees               13130 non-null  float64
dtypes: float64(3), object(5)
memory usage: 923.2+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 5466 entries, 0 to 11799
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   university_name  5466 non-null   object 
 1   college_link     5466 non-null   object 
 2   duration         5466 non-null   object 
 3   fees             5466 non

## 6. Merge Both Datasets

Standardize university names (lowercase, strip whitespace) and merge both datasets using outer join on university_name, program, fees, and duration.


In [111]:
df1_final["university_name"] = df1_final["university_name"].str.strip().str.lower()
df2_final["university_name"] = df2_final["university_name"].str.strip().str.lower()
df_merged = df1_final.merge(
    df2_final,
    on=["university_name", "program", "fees", "duration"],
    how="outer"
)


h 

In [112]:

df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18596 entries, 0 to 18595
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   program            18596 non-null  object 
 1   course_languageEn  13130 non-null  object 
 2   duration           18596 non-null  object 
 3   university_name    18596 non-null  object 
 4   ielts              13130 non-null  float64
 5   toefl              13130 non-null  float64
 6   gpa                13130 non-null  object 
 7   fees               18596 non-null  float64
 8   college_link       5466 non-null   object 
dtypes: float64(3), object(6)
memory usage: 1.3+ MB


In [ ]:
df_merged.head()

<bound method NDFrame.head of                                          program course_languageEn duration  \
0      Bachelor of Business Administration (BBA)               NaN  3 Years   
1                        Bachelor of Engineering               NaN  4 Years   
2                             MSc Energy Storage           English  2 Years   
3                 MSc Sustainable Energy Systems           English  2 Years   
4         Master of Business Administration(MBA)               NaN  2 Years   
...                                          ...               ...      ...   
18591                                 BSc iology           English  4 Years   
18592                      PHD Nurse Anesthetist           English  3 Years   
18593            Bachelor of Technology (B.Tech)               NaN  4 Years   
18594                    Bachelor of Engineering               NaN  4 Years   
18595            Bachelor of Technology (B.Tech)               NaN  4 Years   

                     

## 7. Handle Missing Values and Finalize Dataset

### 7.1 Save Initial Merged Dataset
Save the merged dataset to Excel for backup.

### 7.2 Fill Missing Test Scores
Fill missing IELTS and TOEFL scores with their respective mean values.

### 7.3 Save Final Cleaned Dataset
Export the final cleaned dataset to Excel file.

In [114]:
df_merged.to_excel('..\\data\\all_programs_cleaned.xlsx', index=False)

In [116]:
df_merged2 = pd.read_excel('..\\data\\all_programs_cleaned.xlsx')
df_merged2.head()


,program,course_languageEn,duration,university_name,ielts,toefl,gpa,fees,college_link
0,Bachelor of Business Administration (BBA),English,3 Years,"a c kunhumon haji memorial ica college, thrissur",6.0,92.0,2,650.60,https://www.collegesearch.in/colleges/a-c-kunh...
1,Bachelor of Engineering,English,4 Years,"aalim muhammed salegh college of engineering, ...",NaN,NaN,NaN,2650.60,https://www.collegesearch.in/colleges/aalim-mu...
2,MSc Energy Storage,English,2 Years,aalto university,6.0,92.0,0,38000.00,https://www.ulster.ac.uk/courses/202526/energy...
3,MSc Sustainable Energy Systems,English,2 Years,aalto university,6.0,92.0,0,38000.00,https://www.ulster.ac.uk/courses/202526/energy...
4,Master of Business Administration(MBA),English,2 Years,"aam institute of management and research, chennai",NaN,NaN,NaN,4518.07,https://www.collegesearch.in/colleges/aam-inst...


In [118]:
df_merged2.fillna({
    "ielts": df_merged2["ielts"].mean(),
    "toefl": df_merged2["toefl"].mean()
}, inplace=True)
df_merged2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18596 entries, 0 to 18595
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   program            18596 non-null  object 
 1   course_languageEn  13159 non-null  object 
 2   duration           18596 non-null  object 
 3   university_name    18596 non-null  object 
 4   ielts              18596 non-null  float64
 5   toefl              18596 non-null  float64
 6   gpa                13133 non-null  object 
 7   fees               18596 non-null  float64
 8   college_link       7392 non-null   object 
dtypes: float64(3), object(6)
memory usage: 1.3+ MB


In [119]:

df_merged2.to_excel('..\\data\\all_programs_cleaned.xlsx', index=False)
